In [50]:
import pandas as pd
import numpy as np
import glob
import os

Evictions data is from 2017-2020, althought there are very few 2020 evictions due to moritoriums.  Because Census tract level data isn't available for 2019, 2018 values 5 year estimates are used throughout.  Until at least 2019 census tract data is available, 2018 data is sufficient.

In [51]:
file_list =[]
for file in glob.glob(r"census_data_tract\main_census_folder\*.csv"):
    file_list.append(file)

In [52]:
file_list

['census_data_tract\\main_census_folder\\age_sex.csv',
 'census_data_tract\\main_census_folder\\demographics_housing.csv',
 'census_data_tract\\main_census_folder\\financial_characteristics.csv',
 'census_data_tract\\main_census_folder\\housing_characteristics.csv',
 'census_data_tract\\main_census_folder\\median_gross_rent.csv',
 'census_data_tract\\main_census_folder\\median_income.csv',
 'census_data_tract\\main_census_folder\\monthly_housing_costs.csv',
 'census_data_tract\\main_census_folder\\pop_data.csv',
 'census_data_tract\\main_census_folder\\poverty.csv',
 'census_data_tract\\main_census_folder\\race.csv',
 'census_data_tract\\main_census_folder\\snap.csv',
 'census_data_tract\\main_census_folder\\urban_rural.csv']

In [53]:
master_df = pd.DataFrame()
for file in file_list:
    if master_df.empty:
        master_df = pd.read_csv(file,header=1,index_col='id')
    else:
        current_columns=master_df.columns
        print(file)
        temp_df = pd.read_csv(file,header=1,index_col='id')
        temp_df=temp_df.drop(columns=current_columns,errors='ignore')
        master_df = master_df.join(other=temp_df,how='left')

census_data_tract\main_census_folder\demographics_housing.csv
census_data_tract\main_census_folder\financial_characteristics.csv
census_data_tract\main_census_folder\housing_characteristics.csv
census_data_tract\main_census_folder\median_gross_rent.csv
census_data_tract\main_census_folder\median_income.csv
census_data_tract\main_census_folder\monthly_housing_costs.csv
census_data_tract\main_census_folder\pop_data.csv
census_data_tract\main_census_folder\poverty.csv
census_data_tract\main_census_folder\race.csv
census_data_tract\main_census_folder\snap.csv
census_data_tract\main_census_folder\urban_rural.csv


In [54]:
#an abusrd number of columns to manually sort.
master_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4918 entries, 1400000US36001000100 to 1400000US36123150500
Columns: 3029 entries, Geographic Area Name to Total!!Not defined for this file
dtypes: float64(6), int64(1468), object(1555)
memory usage: 113.8+ MB


In [55]:
#remove extraneous columns
remove_string_list = ['Margin of Error','Total population!!AGE!!','Total population!!SELECTED AGE CATEGORIES!!','Male!!Total population',
                      'Female!!Total population','Occupied housing units!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS',
                      'Occupied housing units!!MONTHLY HOUSING COSTS!!',
                      'Occupied housing units!!MONTHLY HOUSING COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS',
                     'YEAR STRUCTURE BUILT','Total housing units','OCCUPANTS PER ROOM','VALUE!!Owner-occupied units!!','HOUSE HEATING FUEL',
                      'SELECTED MONTHLY OWNER COSTS','VEHICLES AVAILABLE','GROSS RENT','FAMILY INCOME BY NUMBER OF EARNERS',
                      'NONFAMILY HOUSEHOLDS','HOUSEHOLD INCOME BY RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER','FAMILY INCOME BY FAMILY SIZE',
                      'FAMILIES!!Families','YEAR HOUSEHOLDER MOVED INTO UNIT','Population for whom poverty status is determined',
                      'UNRELATED INDIVIDUALS FOR WHOM POVERTY STATUS IS DETERMINED','DISABILITY STATUS',
                      'Native Hawaiian and Other Pacific Islander alone',
                      'SNAP!!Households!!RACE', 'Households!!RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER',
                      'no husband present','no wife present','With children','Married-couple family',
                      'Families!!1 worker in past 12 months','WORK STATUS!!Families!!','Urban!!Inside urbanized',
                      'HOUSEHOLD INCOME BY AGE OF HOUSEHOLDER!!','SNAP!!WORK STATUS!!Families','MORTGAGE STATUS'
                     ]
for string in remove_string_list:
    remove_cols=master_df.columns[master_df.columns.str.contains(string)]
    master_df=master_df.drop(columns=remove_cols)

In [56]:
len(master_df.columns)

309

In [57]:
master_df.columns[50:100]

Index(['Percent Estimate!!RACE!!Total population!!One race!!Asian',
       'Estimate!!RACE!!Total population!!One race!!Asian!!Asian Indian',
       'Percent Estimate!!RACE!!Total population!!One race!!Asian!!Asian Indian',
       'Estimate!!RACE!!Total population!!One race!!Asian!!Chinese',
       'Percent Estimate!!RACE!!Total population!!One race!!Asian!!Chinese',
       'Estimate!!RACE!!Total population!!One race!!Asian!!Filipino',
       'Percent Estimate!!RACE!!Total population!!One race!!Asian!!Filipino',
       'Estimate!!RACE!!Total population!!One race!!Asian!!Japanese',
       'Percent Estimate!!RACE!!Total population!!One race!!Asian!!Japanese',
       'Estimate!!RACE!!Total population!!One race!!Asian!!Korean',
       'Percent Estimate!!RACE!!Total population!!One race!!Asian!!Korean',
       'Estimate!!RACE!!Total population!!One race!!Asian!!Vietnamese',
       'Percent Estimate!!RACE!!Total population!!One race!!Asian!!Vietnamese',
       'Estimate!!RACE!!Total populati

In [58]:
master_df.columns[master_df.columns.str.contains('Estimate!!Total!!WORK STATUS!!')]

Index(['Estimate!!Total!!WORK STATUS!!Families'], dtype='object')

In [59]:
#manually selected traditional key indicators linked to demographics, income and family status
key_variables = ['Geographic Area Name','Estimate!!Total!!Total population',
                 'Estimate!!Total!!Total population!!SUMMARY INDICATORS!!Median age (years)',
                 'Estimate!!Total!!Total population!!SUMMARY INDICATORS!!Sex ratio (males per 100 females)',
                 'Estimate!!Total!!Total population!!SUMMARY INDICATORS!!Age dependency ratio',
                 'Estimate!!Occupied housing units!!Occupied housing units',
                 'Estimate!!Owner-occupied housing units!!Occupied housing units',
                 'Estimate!!Renter-occupied housing units!!Occupied housing units',
                 'Estimate!!Total!!Households!!POVERTY STATUS IN THE PAST 12 MONTHS!!Below poverty level',
                 'Estimate!!Total!!Households!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2018 INFLATION-ADJUSTED DOLLARS)!!Median income (dollars)',
                 'Estimate!!Households receiving food stamps/SNAP!!Households!!POVERTY STATUS IN THE PAST 12 MONTHS!!Below poverty level',
                 'Estimate!!Total!!WORK STATUS!!Families',
                 'Estimate!!Total!!White alone',
                 'Estimate!!Total!!Black or African American alone',
                 'Estimate!!Total!!American Indian and Alaska Native alone',
                 'Estimate!!Total!!Asian alone',
                 'Estimate!!Total!!Some other race alone',
                 'Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)'
                ]

In [60]:
final_df =master_df[key_variables]

In [61]:
final_df.columns

Index(['Geographic Area Name', 'Estimate!!Total!!Total population',
       'Estimate!!Total!!Total population!!SUMMARY INDICATORS!!Median age (years)',
       'Estimate!!Total!!Total population!!SUMMARY INDICATORS!!Sex ratio (males per 100 females)',
       'Estimate!!Total!!Total population!!SUMMARY INDICATORS!!Age dependency ratio',
       'Estimate!!Occupied housing units!!Occupied housing units',
       'Estimate!!Owner-occupied housing units!!Occupied housing units',
       'Estimate!!Renter-occupied housing units!!Occupied housing units',
       'Estimate!!Total!!Households!!POVERTY STATUS IN THE PAST 12 MONTHS!!Below poverty level',
       'Estimate!!Total!!Households!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2018 INFLATION-ADJUSTED DOLLARS)!!Median income (dollars)',
       'Estimate!!Households receiving food stamps/SNAP!!Households!!POVERTY STATUS IN THE PAST 12 MONTHS!!Below poverty level',
       'Estimate!!Total!!WORK STATUS!!Families',
       'Estimate!!Total!!White alo

In [62]:
final_df.head()

,Geographic Area Name,Estimate!!Total!!Total population,Estimate!!Total!!Total population!!SUMMARY INDICATORS!!Median age (years),Estimate!!Total!!Total population!!SUMMARY INDICATORS!!Sex ratio (males per 100 females),Estimate!!Total!!Total population!!SUMMARY INDICATORS!!Age dependency ratio,Estimate!!Occupied housing units!!Occupied housing units,Estimate!!Owner-occupied housing units!!Occupied housing units,Estimate!!Renter-occupied housing units!!Occupied housing units,Estimate!!Total!!Households!!POVERTY STATUS IN THE PAST 12 MONTHS!!Below poverty level,Estimate!!Total!!Households!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2018 INFLATION-ADJUSTED DOLLARS)!!Median income (dollars),Estimate!!Households receiving food stamps/SNAP!!Households!!POVERTY STATUS IN THE PAST 12 MONTHS!!Below poverty level,Estimate!!Total!!WORK STATUS!!Families,Estimate!!Total!!White alone,Estimate!!Total!!Black or African American alone,Estimate!!Total!!American Indian and Alaska Native alone,Estimate!!Total!!Asian alone,Estimate!!Total!!Some other race alone,Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)
id,,,,,,,,,,,,,,,,,,
1400000US36001000100,"Census Tract 1, Albany County, New York",2022,33.6,86.0,71.2,837,216,621,253,29063,169,438,473,1167,88,54,21,412
1400000US36001000200,"Census Tract 2, Albany County, New York",4700,30.7,75.5,42.0,2133,408,1725,688,29470,507,978,621,3583,0,73,0,268
1400000US36001000300,"Census Tract 3, Albany County, New York",5966,37.2,88.5,64.6,2480,914,1566,572,37296,315,1201,2525,1926,17,625,238,628
1400000US36001000401,"Census Tract 4.01, Albany County, New York",2479,65.2,58.3,137.9,1029,673,356,60,75809,0,441,2133,233,15,68,8,127
1400000US36001000403,"Census Tract 4.03, Albany County, New York",4236,37.4,101.1,39.1,2172,1144,1028,306,70488,136,863,3108,484,24,200,104,366


In [63]:
final_df.isna().mean()

Geographic Area Name                                                                                                                 0.0
Estimate!!Total!!Total population                                                                                                    0.0
Estimate!!Total!!Total population!!SUMMARY INDICATORS!!Median age (years)                                                            0.0
Estimate!!Total!!Total population!!SUMMARY INDICATORS!!Sex ratio (males per 100 females)                                             0.0
Estimate!!Total!!Total population!!SUMMARY INDICATORS!!Age dependency ratio                                                          0.0
Estimate!!Occupied housing units!!Occupied housing units                                                                             0.0
Estimate!!Owner-occupied housing units!!Occupied housing units                                                                       0.0
Estimate!!Renter-occupied housing units!!

The errors related to 0 population tend to be locations that actually don't have a a residential area.  These include parks, golf courses and currently uninhabitated islands.  It is problematic that NAs are handled with '-'.

In [64]:
final_df[final_df['Estimate!!Total!!Total population']==0]

,Geographic Area Name,Estimate!!Total!!Total population,Estimate!!Total!!Total population!!SUMMARY INDICATORS!!Median age (years),Estimate!!Total!!Total population!!SUMMARY INDICATORS!!Sex ratio (males per 100 females),Estimate!!Total!!Total population!!SUMMARY INDICATORS!!Age dependency ratio,Estimate!!Occupied housing units!!Occupied housing units,Estimate!!Owner-occupied housing units!!Occupied housing units,Estimate!!Renter-occupied housing units!!Occupied housing units,Estimate!!Total!!Households!!POVERTY STATUS IN THE PAST 12 MONTHS!!Below poverty level,Estimate!!Total!!Households!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2018 INFLATION-ADJUSTED DOLLARS)!!Median income (dollars),Estimate!!Households receiving food stamps/SNAP!!Households!!POVERTY STATUS IN THE PAST 12 MONTHS!!Below poverty level,Estimate!!Total!!WORK STATUS!!Families,Estimate!!Total!!White alone,Estimate!!Total!!Black or African American alone,Estimate!!Total!!American Indian and Alaska Native alone,Estimate!!Total!!Asian alone,Estimate!!Total!!Some other race alone,Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)
id,,,,,,,,,,,,,,,,,,
1400000US36005011000,"Census Tract 110, Bronx County, New York",0,-,-,-,0,0,0,0,-,0,0,0,0,0,0,0,0
1400000US36005016300,"Census Tract 163, Bronx County, New York",0,-,-,-,0,0,0,0,-,0,0,0,0,0,0,0,0
1400000US36005017100,"Census Tract 171, Bronx County, New York",0,-,-,-,0,0,0,0,-,0,0,0,0,0,0,0,0
1400000US36005024900,"Census Tract 249, Bronx County, New York",0,-,-,-,0,0,0,0,-,0,0,0,0,0,0,0,0
1400000US36009940200,"Census Tract 9402, Cattaraugus County, New York",0,-,-,-,0,0,0,0,-,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1400000US36085015400,"Census Tract 154, Richmond County, New York",0,-,-,-,0,0,0,0,-,0,0,0,0,0,0,0,0
1400000US36085990100,"Census Tract 9901, Richmond County, New York",0,-,-,-,0,0,0,0,-,0,0,0,0,0,0,0,0
1400000US36103990100,"Census Tract 9901, Suffolk County, New York",0,-,-,-,0,0,0,0,-,0,0,0,0,0,0,0,0


In [65]:
non_num_col = []
for col in final_df.columns:
    if final_df[col].dtypes == 'object':
        non_num_col.append(col)
non_num_col.remove('Geographic Area Name')

In [66]:
non_num_col

['Estimate!!Total!!Total population!!SUMMARY INDICATORS!!Median age (years)',
 'Estimate!!Total!!Total population!!SUMMARY INDICATORS!!Sex ratio (males per 100 females)',
 'Estimate!!Total!!Total population!!SUMMARY INDICATORS!!Age dependency ratio',
 'Estimate!!Total!!Households!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2018 INFLATION-ADJUSTED DOLLARS)!!Median income (dollars)']

In [67]:
error_cols=[]
for col in non_num_col:
    try:
        final_df[col]=final_df[col].replace('-',np.nan).astype(float)
    except:
        error_cols.append(col)

C:\Users\Public\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [68]:
error_cols

['Estimate!!Total!!Households!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2018 INFLATION-ADJUSTED DOLLARS)!!Median income (dollars)']

In [69]:
final_df[error_cols[0]]=final_df[error_cols[0]].str.replace('+','')
final_df[error_cols[0]]=final_df[error_cols[0]].str.replace(',','')
final_df[error_cols[0]]=final_df[error_cols[0]].replace('-',np.nan)
final_df[error_cols[0]]=final_df[error_cols[0]].astype(float)

C:\Users\Public\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Public\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Public\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

In [70]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4918 entries, 1400000US36001000100 to 1400000US36123150500
Data columns (total 18 columns):
 #   Column                                                                                                                             Non-Null Count  Dtype  
---  ------                                                                                                                             --------------  -----  
 0   Geographic Area Name                                                                                                               4918 non-null   object 
 1   Estimate!!Total!!Total population                                                                                                  4918 non-null   int64  
 2   Estimate!!Total!!Total population!!SUMMARY INDICATORS!!Median age (years)                                                          4848 non-null   float64
 3   Estimate!!Total!!Total population!!SUMMARY INDICATORS!!Sex

In [71]:
final_df.isna().mean()

Geographic Area Name                                                                                                                 0.000000
Estimate!!Total!!Total population                                                                                                    0.000000
Estimate!!Total!!Total population!!SUMMARY INDICATORS!!Median age (years)                                                            0.014233
Estimate!!Total!!Total population!!SUMMARY INDICATORS!!Sex ratio (males per 100 females)                                             0.015860
Estimate!!Total!!Total population!!SUMMARY INDICATORS!!Age dependency ratio                                                          0.015657
Estimate!!Occupied housing units!!Occupied housing units                                                                             0.000000
Estimate!!Owner-occupied housing units!!Occupied housing units                                                                       0.000000
Estima

In [72]:
pop_vars=['Estimate!!Total!!White alone','Estimate!!Total!!Black or African American alone','Estimate!!Total!!American Indian and Alaska Native alone',
         'Estimate!!Total!!Asian alone','Estimate!!Total!!Some other race alone',
          'Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)']
house_vars=['Estimate!!Owner-occupied housing units!!Occupied housing units','Estimate!!Renter-occupied housing units!!Occupied housing units']

In [73]:
new_pop_vars = ['white_pop_%','black_pop_%','native_american_pop_%','asian_pop_%','other_race_%','hispanic_pop_%']
new_house_vars = ['%_owner_housing','%_rental_housing']

In [74]:
pop_vars_dic = dict(zip(pop_vars,new_pop_vars))
house_vars_dic = dict(zip(house_vars,new_house_vars))

In [75]:
pop_vars_dic

{'Estimate!!Total!!White alone': 'white_pop_%',
 'Estimate!!Total!!Black or African American alone': 'black_pop_%',
 'Estimate!!Total!!American Indian and Alaska Native alone': 'native_american_pop_%',
 'Estimate!!Total!!Asian alone': 'asian_pop_%',
 'Estimate!!Total!!Some other race alone': 'other_race_%',
 'Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)': 'hispanic_pop_%'}

In [76]:
for key in pop_vars_dic.keys():
    final_df[pop_vars_dic[key]]=final_df[key]/final_df['Estimate!!Total!!Total population']
for key in house_vars_dic.keys():
    final_df[house_vars_dic[key]]=final_df[key]/final_df['Estimate!!Occupied housing units!!Occupied housing units']

C:\Users\Public\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Public\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [77]:
final_df.columns=final_df.columns.str.replace('Estimate!!Total!!Total population!!SUMMARY INDICATORS!!','')
final_df.columns=final_df.columns.str.replace('Estimate!!Total!!','')
final_df.columns=final_df.columns.str.replace('Estimate!!','')
final_df.columns=final_df.columns.str.replace('Households!!','')
final_df.columns=final_df.columns.str.replace('!!Occupied housing units','')
final_df.columns=final_df.columns.str.replace('WORK STATUS!!','')
final_df.columns=final_df.columns.str.replace('HISPANIC OR LATINO AND RACE!!Total population!!','')

In [78]:
final_df=final_df.rename(columns={'POVERTY STATUS IN THE PAST 12 MONTHS!!Below poverty level':'Households below poverty level (12 months)',
                        'HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2018 INFLATION-ADJUSTED DOLLARS)!!Median income (dollars)':'Median household income (2018 inflation adjusted)',
                        'Households receiving food stamps/SNAP!!POVERTY STATUS IN THE PAST 12 MONTHS!!Below poverty level':'Households receiving food stamps (12 months)',
                        'Hispanic or Latino (of any race)':'Hispanic or Latino'})

In [79]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4918 entries, 1400000US36001000100 to 1400000US36123150500
Data columns (total 26 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   Geographic Area Name                               4918 non-null   object 
 1   Total population                                   4918 non-null   int64  
 2   Median age (years)                                 4848 non-null   float64
 3   Sex ratio (males per 100 females)                  4840 non-null   float64
 4   Age dependency ratio                               4841 non-null   float64
 5   Occupied housing units                             4918 non-null   int64  
 6   Owner-occupied housing units                       4918 non-null   int64  
 7   Renter-occupied housing units                      4918 non-null   int64  
 8   Households below poverty level (12 months)         4918 no

In [80]:
final_df.isna().mean()

Geographic Area Name                                 0.000000
Total population                                     0.000000
Median age (years)                                   0.014233
Sex ratio (males per 100 females)                    0.015860
Age dependency ratio                                 0.015657
Occupied housing units                               0.000000
Owner-occupied housing units                         0.000000
Renter-occupied housing units                        0.000000
Households below poverty level (12 months)           0.000000
Median household income (2018 inflation adjusted)    0.025417
Households receiving food stamps (12 months)         0.000000
Families                                             0.000000
White alone                                          0.000000
Black or African American alone                      0.000000
American Indian and Alaska Native alone              0.000000
Asian alone                                          0.000000
Some oth

In [81]:
final_df.head()

,Geographic Area Name,Total population,Median age (years),Sex ratio (males per 100 females),Age dependency ratio,Occupied housing units,Owner-occupied housing units,Renter-occupied housing units,Households below poverty level (12 months),Median household income (2018 inflation adjusted),...,Some other race alone,Hispanic or Latino,white_pop_%,black_pop_%,native_american_pop_%,asian_pop_%,other_race_%,hispanic_pop_%,%_owner_housing,%_rental_housing
id,,,,,,,,,,,,,,,,,,,,,
1400000US36001000100,"Census Tract 1, Albany County, New York",2022,33.6,86.0,71.2,837,216,621,253,29063.0,...,21,412,0.233927,0.577151,0.043521,0.026706,0.010386,0.203759,0.258065,0.741935
1400000US36001000200,"Census Tract 2, Albany County, New York",4700,30.7,75.5,42.0,2133,408,1725,688,29470.0,...,0,268,0.132128,0.762340,0.000000,0.015532,0.000000,0.057021,0.191280,0.808720
1400000US36001000300,"Census Tract 3, Albany County, New York",5966,37.2,88.5,64.6,2480,914,1566,572,37296.0,...,238,628,0.423232,0.322829,0.002849,0.104760,0.039893,0.105263,0.368548,0.631452
1400000US36001000401,"Census Tract 4.01, Albany County, New York",2479,65.2,58.3,137.9,1029,673,356,60,75809.0,...,8,127,0.860428,0.093990,0.006051,0.027430,0.003227,0.051230,0.654033,0.345967
1400000US36001000403,"Census Tract 4.03, Albany County, New York",4236,37.4,101.1,39.1,2172,1144,1028,306,70488.0,...,104,366,0.733711,0.114259,0.005666,0.047214,0.024551,0.086402,0.526703,0.473297


In [83]:
final_df.to_csv('cleaned_census_data.csv')